In [1]:
import gradio as gr


In [4]:
# 🔧 한글 폰트 문제 해결을 위한 설정
# 이 코드를 메인 코드 실행 전에 먼저 실행해주세요!

import subprocess
import sys

def install_korean_fonts():
    """한글 폰트 설치 및 설정"""
    try:
        # 시스템 업데이트 및 폰트 설치
        commands = [
            "sudo apt-get update -y",
            "sudo apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra",
            "sudo fc-cache -fv"
        ]
        
        for cmd in commands:
            print(f"실행 중: {cmd}")
            result = subprocess.run(cmd.split(), capture_output=True, text=True)
            if result.returncode == 0:
                print("✅ 성공!")
            else:
                print(f"⚠️ 경고: {result.stderr}")
        
        print("\n🎉 한글 폰트 설치 완료!")
        print("이제 메인 앱을 실행해주세요.")
        
    except Exception as e:
        print(f"❌ 폰트 설치 중 오류: {e}")
        print("기본 설정으로 진행합니다.")

# 폰트 설치 실행
install_korean_fonts()

# matplotlib 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 폰트 캐시 재생성 (최신 matplotlib 버전 호환)
try:
    # 최신 버전
    fm.fontManager.__init__()
    print("✅ 폰트 캐시 재생성 완료!")
except:
    try:
        # 이전 버전 호환
        fm._rebuild()
        print("✅ 폰트 캐시 재생성 완료!")
    except:
        print("⚠️ 폰트 캐시 재생성 건너뜀")

# 사용 가능한 폰트 확인
available_fonts = [f.name for f in fm.fontManager.ttflist]
korean_fonts = [font for font in available_fonts if any(keyword in font.lower() for keyword in ['nanum', 'gothic', 'malgun'])]

print(f"사용 가능한 한글 폰트: {korean_fonts[:5] if korean_fonts else '없음'}")

# 나눔고딕 폰트 설정
if korean_fonts:
    font_to_use = korean_fonts[0]
    plt.rcParams['font.family'] = font_to_use
    print(f"✅ 한글 폰트 설정 완료: {font_to_use}")
else:
    plt.rcParams['font.family'] = 'DejaVu Sans'
    print("⚠️ 한글 폰트를 찾을 수 없어 기본 폰트 사용")

plt.rcParams['axes.unicode_minus'] = False

# 테스트 출력
fig, ax = plt.subplots(figsize=(6, 4))
ax.text(0.5, 0.5, '한글 테스트: 안녕하세요! 🎉', 
        fontsize=16, ha='center', va='center')
ax.set_title('폰트 테스트')
plt.show()
plt.close()

print("\n✨ 설정 완료! 이제 메인 앱을 실행하세요!")

실행 중: sudo apt-get update -y
✅ 성공!
실행 중: sudo apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra
⚠️ 경고: E: Package 'fonts-nanum' has no installation candidate
E: Unable to locate package fonts-nanum-coding
E: Unable to locate package fonts-nanum-extra

실행 중: sudo fc-cache -fv
✅ 성공!

🎉 한글 폰트 설치 완료!
이제 메인 앱을 실행해주세요.
✅ 폰트 캐시 재생성 완료!
사용 가능한 한글 폰트: 없음
⚠️ 한글 폰트를 찾을 수 없어 기본 폰트 사용


<Figure size 600x400 with 1 Axes>


✨ 설정 완료! 이제 메인 앱을 실행하세요!


In [ ]:
import gradio as gr
import random
import time
import matplotlib.pyplot as plt
import numpy as np

# Matplotlib 한글 폰트 설정 (필요시 주석 해제 및 폰트 이름 수정)
# try:
#     plt.rcParams['font.family'] = 'Malgun Gothic' # Windows
#     # plt.rcParams['font.family'] = 'NanumGothic' # 예: 나눔고딕 (설치 필요)
#     # plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호 깨짐 방지
# except Exception as e:
#     print(f"폰트 설정 중 오류 발생 (무시하고 진행): {e}")


# 경주마 기본 데이터 (이름, 기본 속도, 지구력)
# 속도(speed_base)는 턴당 평균 이동량에 영향을 줍니다.
# RACE_DISTANCE 와 MAX_TURNS_FOR_ANIMATION 에 맞춰 조정 필요.
# 예: 100m를 50턴(10초)에 가려면 턴당 평균 2m 이동.
HORSES_DATA = [
    {"name": "질풍", "speed_base": 2.2, "stamina": 80},
    {"name": "번개", "speed_base": 2.5, "stamina": 70},
    {"name": "천리마", "speed_base": 2.0, "stamina": 90},
    {"name": "유성", "speed_base": 2.3, "stamina": 75},
    {"name": "적토마", "speed_base": 2.1, "stamina": 85},
    {"name": "백마탄왕자", "speed_base": 2.0, "stamina": 82},
]

RACE_DISTANCE = 100  # 경주 총 거리 (미터)

# 애니메이션 및 게임 시간 설정
ANIMATION_UPDATE_INTERVAL = 0.2  # 초 단위, 애니메이션 프레임 간격 (0.2초 권장)
APPROX_GAME_DURATION = 10      # 초 단위, 목표 게임 시간
MAX_TURNS_FOR_ANIMATION = int(APPROX_GAME_DURATION / ANIMATION_UPDATE_INTERVAL) # 약 50턴

def initialize_horses():
    """경주마들의 초기 상태를 설정하고 컨디션을 적용합니다."""
    initialized_horses = []
    for h_data in HORSES_DATA:
        condition = random.uniform(0.9, 1.1)  # 컨디션: 0.9 ~ 1.1 사이 랜덤값
        initialized_horses.append({
            "name": h_data["name"],
            "position": 0.0,
            "speed_factor": h_data["speed_base"] * condition, # 턴당 이동량에 사용될 기본 값
            "stamina": float(h_data["stamina"]),
            "current_speed_modifier": 1.0,
            "finished_time_sec": float('inf'), # 도착 시간 (초 단위)
            "original_stamina": float(h_data["stamina"]) # 원래 지구력 저장
        })
    return initialized_horses

def plot_race_track(horses_status, race_distance, betting_horse_name=None, turn_info=""):
    """말들의 현재 위치를 Matplotlib 플롯으로 시각화합니다."""
    fig, ax = plt.subplots(figsize=(10, max(3, len(horses_status) * 0.6))) # 최소 높이 3, 말 수에 따라 조절
    
    horse_names = [h['name'] for h in horses_status]
    positions = np.array([h['position'] for h in horses_status])
    
    y_coords = np.arange(len(horse_names))
    
    bar_colors = []
    for h in horses_status:
        if h['name'] == betting_horse_name:
            bar_colors.append('orange') # 배팅한 말
        elif h['position'] >= race_distance:
            bar_colors.append('lightgreen') # 도착한 말
        else:
            bar_colors.append('skyblue') # 진행중인 말

    ax.barh(y_coords, positions, color=bar_colors, edgecolor='gray', height=0.5)
    
    for i, horse in enumerate(horses_status):
        name_display = f"🏇 {horse['name']}"
        fontweight = 'bold' if horse['name'] == betting_horse_name else 'normal'
        ax.text(-race_distance * 0.01, y_coords[i], name_display, 
                va='center', ha='right', color='black', fontweight=fontweight, fontsize=9)
        
        if horse['position'] < race_distance:
            ax.text(min(horse['position'] + race_distance * 0.02, race_distance * 0.95), y_coords[i], 
                    f"{horse['position']:.1f}m", va='center', ha='left', fontsize=8)
        else:
            ax.text(race_distance * 1.01, y_coords[i], "🏁", 
                    va='center', ha='left', fontsize=12, color='red')

    ax.set_yticks(y_coords)
    ax.set_yticklabels([''] * len(horse_names)) # Y축 눈금 레이블 숨김 (이름으로 대체)
    ax.set_xlabel("거리 (m)")
    ax.set_xlim(-race_distance * 0.2, race_distance * 1.15) # 이름 표시 공간 및 결승선 너머 공간 확보
    ax.set_title(f"경주 현황판 - {turn_info}")
    ax.axvline(x=race_distance, color='red', linestyle='--', linewidth=2, label="결승선")
    ax.grid(axis='x', linestyle=':', alpha=0.7)
    
    plt.tight_layout()
    return fig

def run_race_simulation_with_animation(bet_on_horse_name):
    """애니메이션과 함께 경주 시뮬레이션을 실행하고 각 턴의 플롯과 로그를 반환합니다."""
    horses = initialize_horses()
    
    initial_log_text = f"## 🏇 경주 시작! (총 {APPROX_GAME_DURATION}초 예상) 🏇\n"
    if bet_on_horse_name:
        initial_log_text += f"선택하신 응원마: **{bet_on_horse_name}**\n"
    initial_log_text += "\n" + "="*40 + "\n"
    
    turn_info_for_plot = f"턴 0/{MAX_TURNS_FOR_ANIMATION}"
    current_plot = plot_race_track(horses, RACE_DISTANCE, bet_on_horse_name, turn_info_for_plot)
    yield current_plot, initial_log_text # 초기 플롯과 로그

    turn = 0
    winner = None
    active_horses_count = len(horses) # 아직 경주 중인 말의 수
    
    # 게임 시간 (10초)에 맞추기 위한 속도 스케일링은 HORSES_DATA의 speed_base를 통해 조절.
    # speed_base는 턴당 이동 거리의 기본값.

    while active_horses_count > 0 and turn < MAX_TURNS_FOR_ANIMATION:
        turn_summary_log = ""

        for horse in horses:
            if horse["position"] >= RACE_DISTANCE:
                continue # 이미 도착한 말은 이동하지 않음

            # 지구력에 따른 속도 저하 로직 (더 민감하게, 짧은 시간에 효과 보이도록)
            stamina_consumption_per_turn = 1.5 # 턴당 지구력 소모량
            horse["stamina"] = max(0, horse["stamina"] - stamina_consumption_per_turn)
            
            # 남은 지구력 비율에 따라 속도 저하
            stamina_ratio = horse["stamina"] / horse["original_stamina"] if horse["original_stamina"] > 0 else 0
            if stamina_ratio < 0.2: # 20% 미만
                 horse["current_speed_modifier"] = max(0.3, horse["current_speed_modifier"] * random.uniform(0.80, 0.90))
            elif stamina_ratio < 0.5: # 50% 미만
                 horse["current_speed_modifier"] = max(0.6, horse["current_speed_modifier"] * random.uniform(0.90, 0.98))
            else: # 50% 이상
                horse["current_speed_modifier"] = min(1.1, horse["current_speed_modifier"] * random.uniform(0.98, 1.02)) # 컨디션 유지 또는 약간 향상

            # 이동 거리 계산
            move_this_turn = random.uniform(0.8, 1.2) * horse["speed_factor"] * horse["current_speed_modifier"]
            horse["position"] += move_this_turn
            horse["position"] = round(min(horse["position"], RACE_DISTANCE + 5.0), 2) # 결승선 약간 너머까지 허용(시각화용)

            if horse["position"] >= RACE_DISTANCE and horse["finished_time_sec"] == float('inf'):
                horse["finished_time_sec"] = (turn + 1) * ANIMATION_UPDATE_INTERVAL # 초 단위 기록
                horse["position"] = RACE_DISTANCE # 정확히 결승선에
                if not winner:
                    winner = horse["name"]
                active_horses_count -= 1
        
        # 현재 턴의 선두 말 정보
        sorted_horses_for_log = sorted(horses, key=lambda h: (-h["position"], h["finished_time_sec"]))
        if sorted_horses_for_log:
            lead_horse = sorted_horses_for_log[0]
            turn_summary_log = f"턴 {turn + 1}/{MAX_TURNS_FOR_ANIMATION} - 선두: **{lead_horse['name']}** ({lead_horse['position']:.1f}m)\n"
        
        turn_info_for_plot = f"턴 {turn + 1}/{MAX_TURNS_FOR_ANIMATION} ({ (turn+1) * ANIMATION_UPDATE_INTERVAL :.1f}초)"
        current_plot = plot_race_track(horses, RACE_DISTANCE, bet_on_horse_name, turn_info_for_plot)
        
        yield current_plot, initial_log_text + turn_summary_log

        time.sleep(ANIMATION_UPDATE_INTERVAL)
        turn += 1
        if active_horses_count == 0 and turn <= MAX_TURNS_FOR_ANIMATION :
            break

    # 최종 결과 로그 생성
    final_log = initial_log_text # 선택마 정보 포함된 초기 로그 사용
    final_log += f"\n## 🏁🏁 경주 종료! (총 경주 시간: {turn * ANIMATION_UPDATE_INTERVAL:.1f}초) 🏁🏁\n"

    betting_result_message = ""
    if bet_on_horse_name:
        if winner == bet_on_horse_name:
            betting_result_message = f"🎉 축하합니다! **{bet_on_horse_name}** (이)가 우승하여 배팅에 성공했습니다! 🎉\n"
        elif winner:
            betting_result_message = f"아쉽지만 **{bet_on_horse_name}** (은)는 우승하지 못했습니다. 우승마는 **{winner}** 입니다.\n"
        else: # 우승자가 없는 경우 (예: 시간 초과)
             betting_result_message = f"아쉽지만 **{bet_on_horse_name}** (은)는 우승하지 못했습니다. 이번 경주에는 우승마가 없습니다.\n"

    final_log += betting_result_message

    if winner:
        final_log += f"🏆 오늘의 우승마는 **{winner}** 입니다! 🏆\n"
    elif turn >= MAX_TURNS_FOR_ANIMATION:
        final_log += "최대 시간에 도달하여 경주가 종료되었습니다.\n"
    else: 
        final_log += "모든 말이 결승선을 통과했습니다.\n"

    final_ranking = sorted(horses, key=lambda h: (h["finished_time_sec"], -h["position"]))
    final_log += "\n### --- 最終 順位 ---\n"
    for i, hr in enumerate(final_ranking):
        time_record_str = f"{hr['finished_time_sec']:.2f}초" if hr['finished_time_sec'] != float('inf') else "미도착"
        final_log += f"**{i + 1}위**. **{hr['name']}** - {time_record_str} (최종위치: {hr['position']:.1f}m)\n"
    
    final_plot = plot_race_track(horses, RACE_DISTANCE, bet_on_horse_name, f"최종 결과 ({turn * ANIMATION_UPDATE_INTERVAL:.1f}초)")
    yield final_plot, final_log

# Gradio 인터페이스 정의
with gr.Blocks(title="🏇 사이버 경마장 (애니메이션 & 배팅) 🏇", theme=gr.themes.Soft()) as app:
    gr.Markdown("# 🏇 사이버 경마장 🏇")
    gr.Markdown("응원할 말을 선택하고 경주 시작 버튼을 누르세요! 약 10초간 박진감 넘치는 경주가 펼쳐집니다.")

    horse_names_list = [h["name"] for h in HORSES_DATA]
    
    with gr.Row():
        betting_choice = gr.Radio(
            horse_names_list, 
            label="🥇 응원마 선택", 
            value=horse_names_list[0] if horse_names_list else None,
            interactive=True
        )
        start_button = gr.Button("경주 시작!", variant="primary")

    with gr.Row():
        plot_output = gr.Plot(label="경주 현황판")
        text_output = gr.Markdown(value="경마장에 오신 것을 환영합니다! 응원마를 선택하고 경주를 시작하세요.")

    def start_race_wrapper(bet_choice):
        # 경주 시작 시 배팅 선택 비활성화 (선택적)
        # betting_choice.interactive = False # 이 방식은 Gradio에서 직접적으로 지원하지 않음. 업데이트로 처리.
        
        # 시뮬레이션 실행
        # yield from run_race_simulation_with_animation(bet_choice) # 이렇게 하면 안됨
        
        # 아래처럼 각 yield 값을 받아서 다시 yield 해야 함
        for plot, text in run_race_simulation_with_animation(bet_choice):
            yield plot, text #, gr.Radio.update(interactive=False) # 버튼 비활성화 시도
        
        # 경주 종료 후 다시 활성화 (선택적)
        # yield plot_output.value, text_output.value #, gr.Radio.update(interactive=True) # 마지막 값 유지 후 버튼 활성화

    # start_button.click의 inputs/outputs에 맞게 함수 수정이 필요.
    # run_race_simulation_with_animation 자체가 제너레이터이므로 그대로 사용 가능.
    # 컴포넌트의 interactive 속성을 click 내에서 직접 바꾸는 것은 복잡.
    # 대신, 출력을 여러 컴포넌트에 한 번에 할 수 있음.

    start_button.click(
        fn=run_race_simulation_with_animation, 
        inputs=[betting_choice], 
        outputs=[plot_output, text_output]
    )

if __name__ == '__main__':
    app.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


/tmp/ipykernel_2710/1953481240.py:93: UserWarning: Glyph 44144 (\N{HANGUL SYLLABLE GEO}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_2710/1953481240.py:93: UserWarning: Glyph 47532 (\N{HANGUL SYLLABLE RI}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_2710/1953481240.py:93: UserWarning: Glyph 44221 (\N{HANGUL SYLLABLE GYEONG}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_2710/1953481240.py:93: UserWarning: Glyph 51452 (\N{HANGUL SYLLABLE JU}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_2710/1953481240.py:93: UserWarning: Glyph 54788 (\N{HANGUL SYLLABLE HYEON}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_2710/1953481240.py:93: UserWarning: Glyph 54889 (\N{HANGUL SYLLABLE HWANG}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_2710/1953481240.py:93: UserWarning: Glyph 54032 (\N{HANGUL SYLLABLE PAN}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_2710/1953481240.py:93